In [1]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [2]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [3]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [4]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


In [13]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a senior computer scientist and a research scientist with more than 30 years of explerience in conducting research experiments around novel appproach in computer Linguistics and Natural Language processing.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


In [14]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [15]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [16]:
system_prompt = "You are an CS consultant and research scientist that excels at giving advice and helps in brainstorming "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout ideas:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are an CS consultant and research scientist that excels at giving advice and helps in brainstorming 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout ideas:

 {text}[/INST]


In [17]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [22]:
text = "I want to some experiment and test my own hypothesis. i.e Can LLM come up with novel ideas and methodology in research areas and analytical findings?. Thats my hypothesis. Now i dont know what to do next. Can you suggest some potential experimentation that helps for me?"

In [23]:
response = llm_chain.run(text)
print(response)

  Ah, an ambitious aspiring researcher! *adjusts glasses* I must say, proposing a hypothesis and seeking experiments to test it is a commendable start. However, I must offer some refinements to your approach to ensure a scientifically rigorous and effective examination of your hypothesis.

To begin with, your hypothesis should be more specific and detailed. Instead of simply stating "Can LLM come up with novel ideas and methodology in research areas?", consider a more focused statement like: "I hypothesize that LLM can generate novel and innovative ideas in [specific research area] by leveraging its language models and neural networks."

Once you have a more specific hypothesis, we can move on to designing experiments to test it. Here are a few potential approaches you could consider:

1. Idea Generation:
In this experiment, you could feed a large dataset of texts (e.g., scientific articles, patents, or even chat logs) into your LLM, and then analyze the generated ideas for novelty and

In [21]:
response = llm_chain.run(text)
print(response)

  Ah, I see! 🤔 As a CS consultant and research scientist, I'm glad to hear that you're interested in exploring the potential of Large Language Models (LLMs) in generating novel ideas and methodologies in research areas. This is indeed an exciting area of exploration, with many possibilities for creative and innovative applications! 💡

To help you in your quest, here's a step-by-step guide on how you can experiment and test your own hypotheses on the capabilities of LLMs:

1. Define Your Research Area: 🕔
Start by identifying a specific research area that you're interested in exploring with LLMs. This could be anything from natural language processing, computer vision, or even multi-modal modeling. By focusing on a specific area, you can tailor your experiments to the unique challenges and opportunities present in that domain.
2. Choose an LLM: 🤖
Next, select an appropriate LLM that aligns with your research area. There are numerous LLMs available, each with their strengths and weaknesse

In [19]:
response = llm_chain.run(text)
print(response)

  As a compassionate and understanding CS consultant and research scientist, I understand that you may be feeling stuck and unsure about how to proceed with your research on Large Language Models (LLMs). Here's a step-by-step guide to help you navigate this challenge:

1. Identify your goals: Start by clearly defining what you want to achieve with your research on LLMs. Are you trying to improve existing models or develop new ones? Are you interested in exploring specific applications or domains? Once you have a clear goal in mind, it will be easier to determine the most effective approaches and strategies to take.
2. Conduct a literature review: Review existing research on LLMs to gain a deeper understanding of the current state-of-the-art in the field. Identify key concepts, methodologies, and challenges that are relevant to your research goals. This will help you identify potential avenues for experimentation and innovation.
3. Experiment with different techniques: Based on your rev